# MACHINE LEARNING - Projeto de Classificacao de Cervejas parte 2 - Aplicacao dos 2 melhores modelos de classificacao em dataset apos pre-processamento + Ajuste de Parametros + Feature Selection

. OBJETIVO: Aplicar modelos de machine learning em um problema de classificação com o intuito de melhorar a assertividade ao classificar os tipos de cervejas de acordo com características fornecidas no dataset.
. ETAPAS:

1. Carregamento e processamento completo do dataset
2. Após aplicação de 4 modelos de classificação para fins comparativos, otimização e aplicação dos dois melhores modelos de machine learning: KNN e Decision Tree.
3. Identificação das variáveis que mais impactam nos resultados da análise dos algoritmos.

<b>CARREGANDO DATASET PRÉ-PROCESSADO<b>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dados = pd.read_csv('Beerdataset_ready.csv')
dados.head(3)

,Unnamed: 0,All Grain,BIAB,Partial Mash,extract,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,PitchRate,PrimaryTemp
0,2,0,0,0,1,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.047,70.0,1.5,0,0.677102,20.0
1,3,1,0,0,0,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.047,70.0,1.5,0,0.677102,20.0
2,5,1,0,0,0,10,24.61,1.055,1.013,5.58,40.12,8.00,29.34,70,1.047,79.0,1.5,0,1.000000,20.0


<b>MACHINE LEARNING: Bibliotecas, variaveis target/preditora, modelos<b>

In [3]:
#Importando bibliotecas necessarias:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Criando variaveis preditoras/target:
y = dados['StyleID']
x = dados.drop('StyleID', axis=1)

<b>KNN: APLICAÇÃO DE MODELOS: COM E SEM CROSS VALIDATION<b>

In [6]:
#Normalizando dados:
normalizadorKNN = MinMaxScaler(feature_range=(0,1))
x_normKNN = normalizadorKNN.fit_transform(x)

#Bases de treino e teste:
x_treino, y_treino, x_teste, y_teste = train_test_split(x_normKNN, y, test_size=0.3, random_state=15)

In [7]:
#aplicando modelos sem SKfold:
def modelosML2(a,b):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.preprocessing import MinMaxScaler
    x = a
    y = b
    knn = KNeighborsClassifier()
    tree = DecisionTreeClassifier()
    knn.fit(x,y), 
    tree.fit(x,y)
    resultKnn = knn.score(x,y)
    resultTree = tree.score(x,y)
    print('Resultado KNN:', resultKnn, 'Resultado Decision Tree:', resultTree)

modelosML2(x,y)

Resultado KNN: 0.5297820686540199 Resultado Decision Tree: 1.0


In [8]:
#aplicando modelos com cross validation:
def modelosML1(a,b):
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import StratifiedKFold
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    x=a
    y=b
    skfold = StratifiedKFold(n_splits=5)
    knn = KNeighborsClassifier()
    tree = DecisionTreeClassifier()
    result_knn = cross_val_score(knn,x,y,cv = skfold)
    result_tree = cross_val_score(tree,x,y,cv = skfold)
    print('Resultado KNN:', result_knn.mean(), 'Resultado Decision Tree:', result_tree.mean())

modelosML1(x,y)

Resultado KNN: 0.022133088291843777 Resultado Decision Tree: 0.2430797722421177


<b>AJUSTE DE PARÂMETROS E OTIMIZAÇÃO DO MELHOR MODELO - DECISION TREE<b>

In [9]:
#Importando bibliotecas para o GridSearchCV:
import numpy as np
from sklearn.model_selection import GridSearchCV

#Criando os valores para ajuste de parametros da arvore:
num_splits = np.array([2,3,4,5,6,7,8,9])
max_nivel = np.array([3,4,5,6,])
algoritmo = ['gini', 'entropy']    #escolhe entre calculo via gini ou entropia para obtencao do melhor resultado
valores_grid = {'min_samples_split': num_splits, 'max_depth': max_nivel, 'criterion': algoritmo}

In [10]:
#Instanciando o modelo: 
modelo = DecisionTreeClassifier()
#Criando os grids:
gridTree = GridSearchCV(estimator=modelo, param_grid=valores_grid, cv=5)
gridTree.fit(x,y)
#Imprimindo melhores parametros:
print('Minimo Split:', gridTree.best_estimator_.min_samples_split)
print('Maxima Profundidade:', gridTree.best_estimator_.max_depth)
print('Algoritmo Escolhido:', gridTree.best_estimator_.criterion)
print('Resultado da Acuracia:', gridTree.best_score_)

Minimo Split: 2
Maxima Profundidade: 5
Algoritmo Escolhido: gini
Resultado da Acuracia: 0.5283968962828157


<b>FEATURE SELECTION: IDENTIFICANDO FEATURES MAIS IMPACTANTES DO DATASET<b>

In [11]:
#Importando bibliotecas
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest  #seleciona variaveis com mais score

In [12]:
#Selecionando 3 features mais relevantes:
quiQuadrado = SelectKBest(score_func=chi2, k=3)
melhoresQui = quiQuadrado.fit_transform(x,y)

ValueError: Input X must be non-negative.

<b>Considerando o erro acima de qui-quadrado (pois nao calcula com numeros negativos), vamos optar por aplicar o algoritmo  f_classif que considera tanto numeros postitivos quanto negativos no calculo de identificacao das features.

In [ ]:
#Selecionando 3 features mais relevantes com f_classif:
fClass = SelectKBest(score_func=f_classif, k=3)
melhoresClass = fClass.fit_transform(x,y)

#Printando resultados:
print('Scores de Cada Feature:', fClass.scores_)
print('Resultado da Transformacao:', melhoresClass)

In [ ]:
#AS FEATURES 5, 8 e 12 SAO AS MAIS RELEVANTES PARA O DATASET.
x.head(2)

In [ ]:
#FEATURES 5, 8 e 12.
x[['Size(L)', 'ABV', 'BoilTime']].head()

<b>APLICANDO 'GABARITO' PARA IDENTIFICAR PROPORÇÃ DE FALSO POSITIVOS/NEGATIVOS - CONFUSION MATRIX<b>

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
modelo = DecisionTreeClassifier()
modelo.fit(x,y)

In [ ]:
predicao = modelo.predict(x)
predicao

In [ ]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y, predicao)
print(matriz)

# VISUALIZANDO A ÁRVORE DE DECISÃO

In [ ]:
#Visualizando arvore:
modelo.fit(x,y)
fig1 = plt.figure(figsize=(25,20))
fig1 = tree.plot_tree(modelo, filled=True)

In [ ]:
#Salvando arvore:
fig.savefig("decision_tree_graph.png")